In [1]:
# purchase_order_pipeline.py
import pandas as pd
import numpy as np
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings
warnings.filterwarnings('ignore')

# Load data
sales_df = pd.read_csv(r"C:\Users\Vishwa\Desktop\GUVI captone\Dominos\Pizza_Sale - pizza_sales.csv")
ingredients_df = pd.read_csv(r"C:\Users\Vishwa\Desktop\GUVI captone\Dominos\Pizza_ingredients - Pizza_ingredients.csv")

# Preprocess data
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], errors='coerce', dayfirst=True)
daily_sales = sales_df.groupby('order_date')['quantity'].sum()
daily_sales_df = daily_sales.reset_index().rename(columns={'order_date': 'ds', 'quantity': 'y'})

# Train/Test Split
train = daily_sales[:-7]
test = daily_sales[-7:]

# SARIMA Forecast
sarima_model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,7))
sarima_result = sarima_model.fit()
sarima_forecast = sarima_result.forecast(steps=7)
avg_daily_pred = sarima_forecast.mean()

# Generate Pizza Forecast
pizza_ratios = sales_df.groupby('pizza_name_id')['quantity'].sum() / sales_df['quantity'].sum()
pizza_forecast = (pizza_ratios * avg_daily_pred * 7).reset_index()
pizza_forecast.columns = ['pizza_name_id', 'forecasted_quantity']

# Purchase Order Generation
purchase_order = pd.merge(pizza_forecast, ingredients_df, on='pizza_name_id', how='left')
purchase_order['total_ingredient_qty'] = purchase_order['forecasted_quantity'] * purchase_order['Items_Qty_In_Grams']
ingredient_summary = purchase_order.groupby('pizza_ingredients')['total_ingredient_qty'].sum().reset_index()
ingredient_summary.columns = ['Ingredient', 'Total_Required_Grams']
ingredient_summary = ingredient_summary.sort_values(by='Total_Required_Grams', ascending=False)

# Export Purchase Order
ingredient_summary.to_csv(r"C:\Users\Vishwa\Desktop\GUVI captone\Dominos\New Folder\final_purchase_order.csv", index=False)

# Print Summary
print("\n=== Final Purchase Order Generated Successfully ===")
print(ingredient_summary.head(10))



=== Final Purchase Order Generated Successfully ===
    Ingredient  Total_Required_Grams
15     Chicken          19260.888201
51  Red Onions          17124.711245
13   Capocollo          15304.013500
62    Tomatoes          11781.195363
6        Bacon           9952.367732
44   Pepperoni           8624.420357
37   Mushrooms           7379.159865
25      Garlic           6180.497497
58     Spinach           5997.475931
20        Corn           5421.245518
